In [4]:
import codecs
import csv
import re
from Levenshtein import distance
from tqdm import tqdm_notebook as tqdm
DATASET_PATH = '/Users/aleksandr.fedotov/Downloads/dataset_37845_1.txt'
DICTIONARY_PATH = 'data/odict.csv'
DICTIONARY_OPCORPORA_PATH = 'data/dict.opcorpora.txt'

In [2]:
S = 'S'
A = 'A'
V = 'V'
PR = 'PR'
CONJ = 'CONJ'
ADV = 'ADV'
pos_map = {'межд.': ADV,
 'с': S,
 'союз': CONJ,
 'част.': ADV,
 'ж': S,
 'м': S,
 'п': A,
 'мо': S,
 'жо': S,
 'св-нсв':V,
 'н': ADV,
 'вводн.': ADV,
 'нсв': V,
 'мн.': S,
 'св': V,
 'предл.': PR,
 'предик.': ADV,
 'мо-жо': S,
 'со': S,
 'сравн.': ADV,
 'мс-п': ADV,
 'числ.-п': A,
 'числ.': ADV}
pos_map_opcorpora = {
 'NOUN': S,
 'ADVB': ADV,
 'ADJF': A,
 'ADJS': A,
 'COMP': A,
 'VERB': V,
 'INFN': V,
 'PRTF': V,
 'PRTS': A,
 'GRND': V,
 'CONJ': CONJ,
 'INTJ': ADV,
 'PRCL': ADV,
 'PREP': PR,
 'PRED': ADV,
 'NUMR': ADV,
 'NPRO': ADV}

In [22]:
[(p, ws[:20])for p, ws in poses.items()]

[('NOUN',
  ['еж',
   'еж',
   'ежик',
   'елка',
   'елочка',
   'емкость',
   'ера',
   'ерник',
   'ерш',
   'ерш',
   'ершик',
   'ершик',
   'а-конто',
   'а-ось',
   'ааво',
   'аавович',
   'аавовна',
   'аамир',
   'аамирович',
   'аамировна']),
 ('ADVB',
  ['ежиком',
   'абазински',
   'абсолютно',
   'абстрактно',
   'абсурдно',
   'абхазски',
   'авансом',
   'аварски',
   'автоматически',
   'авторски',
   'адекватно',
   'аджарски',
   'адски',
   'адыгейски',
   'ажурно',
   'азартно',
   'азиатски',
   'азовски',
   'айсорски',
   'аккордно']),
 ('ADJF',
  ['ежистый',
   'елочный',
   'емкий',
   'емкостный',
   'ернический',
   'абабковский',
   'абагайтуйский',
   'абагурский',
   'абадзехский',
   'абажурный',
   'абазинский',
   'абайский',
   'абакановский',
   'абаканский',
   'абакумовский',
   'абалаковский',
   'абанский',
   'абатский',
   'абашевский',
   'аббакумовский']),
 ('ADJS',
  ['ежист',
   'емок',
   'емкостен',
   'абажурен',
   'аббревиатурен',
   '

In [11]:
dictionary_opcorpora['если']

('если', 'CONJ')

In [7]:
dictionary_opcorpora['про']

('про', 'S')

In [31]:
dictionary = {}
lines = []
mains = []
with codecs.open(DICTIONARY_PATH, 'r', 'cp1251') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in tqdm(reader):
        row = [r.lower().replace('ё', 'е') for r in row]
        main = row[0]
        pos = pos_map.get(row[1], row[1])
        dictionary[main] = (main, pos)
        mains.append(main)
        if len(row) < 3:
            continue
        for f in row[2:]:
            dictionary[f] = (main, pos)

In [12]:
dictionary_opcorpora = {}
mains_opc = []
poses = {}
with open(DICTIONARY_OPCORPORA_PATH, 'r') as file:
    cur_main = ''
    new_word = True
    for line in tqdm(file):
        if line == '\n':
            continue
        if line[:-1].isdecimal():
            new_word = True
            continue
        line = line.split()
        word = line[0].lower().replace('ё', 'е')
        pos = pos_map_opcorpora[line[1].split(',')[0]]
        if new_word:
            new_word = False
            cur_main = word
            mains_opc.append(cur_main)
            poses[line[1].split(',')[0]] = poses.get(line[1].split(',')[0], []) + [word]
        dictionary_opcorpora[word] = dictionary_opcorpora.get(word, []) + [(cur_main, pos)]

In [18]:
def find_closest(token, mains):
    min_r = 10000
    main_res = ''
    for main in mains:
        d = distance(token, main)
        if d < min_r:
            main_res = main
            min_r = d
    print(token, main_res, min_r)
    return main_res

In [19]:
def solve(dictionary, mains):
    res = ''
    c = 0
    full_c = 0
    with open(DATASET_PATH, 'r') as file:
        for line in file:
            tokens = re.split(r'[\s,\.\?\!]+', line)
            tokens = [t for t in tokens if t != '']
            for token in tokens:
                prev_t = token
                token = token.lower().replace('ё', 'е')
                full_c += 1
                if token not in dictionary:
                    c += 1
                    maybe_main = find_closest(token, mains)
                    main, pos = dictionary[maybe_main][0]
                else:
                    main, pos = dictionary[token][0]
                res += ' ' + prev_t + '{' + main + '=' + pos + '}'
            res += '\n'
    print(f'{c}/{full_c}({int(c / full_c * 100)}%) are unknown')
    return res

In [25]:
res = solve(dictionary_opcorpora, mains_opc)

лпх лех 1
госрасходов городов 4
флоринская лозинская 2
микрофинансов минфинансов 3
однонаправленности направленность 5
асоуп агоп 2
контрпродуктивный непродуктивный 4
рм гм 1
вакцинородственных вакцинированный 6
оюл обл 1
дункан букан 2
кагальнинцы кагальник 3
нудоль рудоль 1
гильермо гиленко 3
капетильо капель 3
изауры заур 2
строу строб 1
сицилий сицилия 1
18/3284(0%) are unknown


In [35]:
def get_res(token, dictionary, mains):
    if token not in dictionary:
        maybe_main = find_closest(token, mains)
        return dictionary[maybe_main][0]
    else:
        return dictionary[token][0]

In [36]:
def solve2():
    res = ''
    c = 0
    full_c = 0
    with open(DATASET_PATH, 'r') as file:
        for line in file:
            tokens = re.split(r'[\s,\.\?\!]+', line)
            tokens = [t for t in tokens if t != '']
            for token in tokens:
                prev_t = token
                token = token.lower().replace('ё', 'е')
                full_c += 1
                main_o, pos = get_res(token, dictionary_opcorpora, mains_opc)
                if token not in dictionary:
                    c += 1
                    main = main_o
                else:
                    main, _ = dictionary[token]
                res += ' ' + prev_t + '{' + main + '=' + pos + '}'
            res += '\n'
    print(f'{c}/{full_c}({int(c / full_c * 100)}%) are unknown')
    return res

In [38]:
res = solve2()

замочаленный замолвленный 2
ммд мед 1
симагин самарин 2
субурбанизации дезурбанизация 4
отельчики котельчик 2
миксобактерии нитробактерия 4
ибор бор 1
природников бродников 3
мошков машков 1
полуразрушенном неразрушенный 6
равность давность 1
остерсунде эстерсунд 2
лютера литера 1
кальвина альвина 1
догнивания догнивая 2
спонсируют анонсирую 3
автосоревнования автовыравнивание 5
автособытию автогонки 5
бернес бернер 1
трезвяк трезвя 1
пейзажики пейзажист 2
маурья марья 1
доимперских имперски 3
сапробиологический агробиологический 2
митрофанушки митрофан 4
еласы власы 1
мешкова машкова 1
психогенетики психогенез 4
сгибовой силовой 2
горбиманией германий 4
визенталь вежеталь 3
нераненый неграненый 1
равноудаленности равноудаленный 4
доули долли 1
305/3513(8%) are unknown


In [39]:
with open('res.txt', 'w') as rfile:
    rfile.write(res)